In [54]:
import pandas as pd
dfcount = pd.read_csv("0667ba91-d9e6-4f8e-b246-fc77ec36ba77.csv")
dfcount

,content_languages,count
0,eng,982966944
1,NaN,585787318
2,rus,107580325
3,zho,90350792
4,deu,86497311
...,...,...
218506,"eng,mal,mlt",1
218507,"ind,san,aze",1
218508,"tat,deu,msa",1
218509,"srp,mlg,sqi",1


In [57]:
from collections import defaultdict

counts = {}
for ix, row in dfcount.query("content_languages.notna()").iterrows():
    lang = row["content_languages"]
    lang = lang.split(",")
    for l in lang:
        if l not in counts.keys():
            counts[l] = 0
        counts[l] += row["count"]

counts = {"id": list(counts.keys()), "n_commoncrawl": list(counts.values())}
counts = pd.DataFrame(counts)
counts

,id,n_commoncrawl
0,eng,1647268067
1,rus,175309697
2,zho,158700611
3,deu,161337878
4,spa,138488278
...,...,...
156,sag,11777
157,sux,2087
158,got,1289
159,kas,637


In [69]:
df = [line.split("\t") for line in open("isocodes.tsv")][1:]
#df[0] = df[0][1:]
df = list(filter(lambda x:len(x)==len(df[0]), df))
df = pd.DataFrame(df[1:], columns=df[0])
df

,ISO 639-2 Code,ISO 639-1 Code,English name of Language,French name of Language,German name of Language\n
0,aar,aa,Afar,afar,Danakil-Sprache\n
1,abk,ab,Abkhazian,abkhaze,Abchasisch\n
2,ace,,Achinese,aceh,Aceh-Sprache\n
3,ach,,Acoli,acoli,Acholi-Sprache\n
4,ada,,Adangme,adangme,Adangme-Sprache\n
...,...,...,...,...,...
502,znd,,Zande languages,"zandé, langues",Zande-Sprachen\n
503,zul,zu,Zulu,zoulou,Zulu-Sprache\n
504,zun,,Zuni,zuni,Zuñi-Sprache\n
505,zxx,,No linguistic content; Not applicable,pas de contenu linguistique; non applicable,Kein linguistischer Inhalt\n


In [70]:
def innerjoin(df1, df2, column1, column2):
    d = {}
    for ix, row in df1.iterrows():
        d[row[column1]] = row
    newdf = df2.copy()
    for ix, row in df2.iterrows():
        if row[column2] in d.keys():
            newdf.append(d[column2] + row)
    return pd.DataFrame(newdf)

innerjoin(counts, df, "id", "ISO 639-1 Code")

,ISO 639-2 Code,ISO 639-1 Code,English name of Language,French name of Language,German name of Language\n
0,aar,aa,Afar,afar,Danakil-Sprache\n
1,abk,ab,Abkhazian,abkhaze,Abchasisch\n
2,ace,,Achinese,aceh,Aceh-Sprache\n
3,ach,,Acoli,acoli,Acholi-Sprache\n
4,ada,,Adangme,adangme,Adangme-Sprache\n
...,...,...,...,...,...
502,znd,,Zande languages,"zandé, langues",Zande-Sprachen\n
503,zul,zu,Zulu,zoulou,Zulu-Sprache\n
504,zun,,Zuni,zuni,Zuñi-Sprache\n
505,zxx,,No linguistic content; Not applicable,pas de contenu linguistique; non applicable,Kein linguistischer Inhalt\n


In [58]:
df = [line.split("\t") for line in open("speakers.tsv")]
df = list(filter(lambda x:len(x) == len(df[0]), df))
columns = [x.strip() for x in df[0]]
df = pd.DataFrame(df[1:], columns=columns)
df = df[["Name", "ISO Alpha-3/5", "Speakers worldwide"]]
columns = {
    'Name': "name", 
    'ISO Alpha-3/5': "id",
    'Speakers worldwide': "n_speakers"
}
df = df.rename(columns=columns).query("n_speakers.notna() and id.notna()")

num_speakers = {}
lang_names = {}
for ix, row in df.iterrows():
    num_speakers[row["id"]] = row["n_speakers"]
    lang_names[row["id"]] = row["name"]

n_speakers = []
l_names = []
for id in counts.id:
    if id in num_speakers.keys():
        n_speakers.append(num_speakers[id])
    else:
        n_speakers.append(pd.NA)
    if id in lang_names.keys():
        l_names.append(lang_names[id])
    else:
        l_names.append(pd.NA)

counts["name"] = l_names
counts["n_speakers"] = n_speakers
counts

,id,n_commoncrawl,name,n_speakers
0,eng,1647268067,<NA>,<NA>
1,rus,175309697,<NA>,<NA>
2,zho,158700611,<NA>,<NA>
3,deu,161337878,<NA>,<NA>
4,spa,138488278,<NA>,<NA>
...,...,...,...,...
156,sag,11777,Sango,5004000
157,sux,2087,<NA>,<NA>
158,got,1289,<NA>,<NA>
159,kas,637,Kashmiri,5600000
